In [1]:
import os.path as osp
import os
import errno
from re import T
import shutil
import numpy as np
import glob
import pathlib
from functools import partial
from multiprocessing import Pool, get_context
import pandas as pd
import numpy as np
areas=glob.glob("data/helix/raw/data/*/")
print('# of areas',len(areas))
print('areas:',areas)
files=glob.glob("data/helix/raw/data/*/*.csv")
len(files)

# of areas 11
areas! ['data/helix/raw/data/2000_lvl_1/', 'data/helix/raw/data/4000_lvl_5_area5/', 'data/helix/raw/data/1225_charleston_lvl2/', 'data/helix/raw/data/4000_lvl_5_area_3/', 'data/helix/raw/data/1225_charleston_lvl1/', 'data/helix/raw/data/2000_lvl4/', 'data/helix/raw/data/helix_hq_lvl2/', 'data/helix/raw/data/2000_lvl2/', 'data/helix/raw/data/big_oakland/', 'data/helix/raw/data/4000_lvl5_area4/', 'data/helix/raw/data/jtc_lvl14/']


687

In [2]:
# labels = { "barrier" : 0, "beam" : 1, "ceiling" : 2, "column" : 3, "door" : 4, "fire equipment" : 5, "fixed object" : 6, "floor" : 7, "glass wall" : 8, "light" : 9, "mechanical element" : 10, "moveable object" : 11, "noise" : 12, "other structural element" : 13, "ramp" : 14, "stair" : 15, "vegetation" : 16, "wall" : 17, "window" : 18 }
# regroupings = { "wall": ["column", "glass wall", "wall"], "ceiling": ["ceiling", "beam"], "clutter": ["fire equipment", "mechanical element", "other structural element", "ramp", "vegetation", "noise"], "object": ["fixed object", "moveable object", "barrier"] }
correspondance=pd.read_csv("./correspondance.csv",usecols=["Classification", "regrouping_int"])
print(f"# large classes {correspondance.Classification.nunique()}")
print(f"# small classes {correspondance.regrouping_int.nunique()}")
correspondance

# large classes 19
# small classes 9


,Classification,regrouping_int
0,1,0
1,2,1
2,3,1
3,4,2
4,5,3
5,6,4
6,7,0
7,8,5
8,9,2
9,10,6


In [3]:
import numpy as np
import pandas as pd
df=pd.read_csv(files[0])
df['X']=df['X'].astype(np.float32)
df['Y']=df['Y'].astype(np.float32)
df['Z']=df['Z'].astype(np.float32)
df['R']=None
df['G']=None
df['B']=None
df['Classification']=df['Classification'].astype(int)
df=df.merge(correspondance,how="left",on="Classification")
df=df[['X','Y','Z','R','G','B','regrouping_int']]
# np.save("data/helix/pointnext/raw/demo.npy",df.values,allow_pickle=True)
df.head(1)

,X,Y,Z,R,G,B,regrouping_int
0,-1.77,-12.07,2.94,None,None,None,2


helix 
object: 0
ceiling: 1
wall: 2
door: 3
clutter: 4
floor: 5
light: 6
stair: 7
window: 8

s3dis {0: 'ceiling'
1: 'floor'
2: 'wall'
3: 'beam'
4: 'column'
5: 'window'
6: 'door'
7: 'chair'
8: 'table'
9: 'bookcase'
10: 'sofa'
11: 'board'
12: 'clutter'}

In [4]:
import os
import numpy as np
from pathlib import Path
chunk_size=500000
nb_files_total=0
output_path="data/helix/pointnext/raw"
for f in files:
    f2=f.replace("data/helix/raw",output_path)    
    df=pd.read_csv(f)
    df['X']=df['X'].astype(np.float32)
    df['Y']=df['Y'].astype(np.float32)
    df['Z']=df['Z'].astype(np.float32)
    df['R']=0#None
    df['G']=0#None
    df['B']=0#None
    df['Classification']=df['Classification'].astype(int)
    s=df.shape[0]
    df=df[df.Classification.notnull()]
    s2=df.shape[0]
    if s!=s2:
        print("WARNING",f,s,s2)
    df=df.merge(correspondance,how="left",on="Classification")
    df=df[['X','Y','Z','R','G','B','regrouping_int']]
    nb_cut=1+(df.shape[0]//chunk_size)
    df['X2'] = pd.cut(df.X, nb_cut, right=False)
    area=f2.split(os.sep)[5]
    file=f2.split(os.sep)[6].replace(".csv","")
    index=0
    for x in df['X2'].unique():
        df_subset = df[(df.X2==x)]
        print(f"{nb_files_total} - {output_path}/Area_{area}_{file}_{index}.npy",df_subset.shape[0])
        np.save(f"{output_path}/Area_{area}_{file}_{index}.npy",df_subset[['X','Y','Z','R','G','B','regrouping_int']].values,allow_pickle=True)
        index += 1
        nb_files_total +=1

0 - data/helix/pointnext/raw/Area_2000_lvl_1_27_0.npy 391670
1 - data/helix/pointnext/raw/Area_2000_lvl_1_27_1.npy 391783
2 - data/helix/pointnext/raw/Area_2000_lvl_1_27_2.npy 297673
3 - data/helix/pointnext/raw/Area_2000_lvl_1_20_0.npy 406718
4 - data/helix/pointnext/raw/Area_2000_lvl_1_20_1.npy 526040
5 - data/helix/pointnext/raw/Area_2000_lvl_1_20_2.npy 406306
6 - data/helix/pointnext/raw/Area_2000_lvl_1_29_0.npy 669480
7 - data/helix/pointnext/raw/Area_2000_lvl_1_29_1.npy 478735
8 - data/helix/pointnext/raw/Area_2000_lvl_1_29_2.npy 565692
9 - data/helix/pointnext/raw/Area_2000_lvl_1_29_3.npy 232710
10 - data/helix/pointnext/raw/Area_2000_lvl_1_29_4.npy 382331
11 - data/helix/pointnext/raw/Area_2000_lvl_1_17_0.npy 350360
12 - data/helix/pointnext/raw/Area_2000_lvl_1_17_1.npy 521519
13 - data/helix/pointnext/raw/Area_2000_lvl_1_17_2.npy 241508
14 - data/helix/pointnext/raw/Area_2000_lvl_1_38_0.npy 457442
15 - data/helix/pointnext/raw/Area_2000_lvl_1_38_1.npy 514938
16 - data/helix/po

In [5]:
np.load("data/helix/pointnext/raw/Area_1225_charleston_lvl2_41_1.npy",allow_pickle=True).shape

(779285, 7)

In [6]:
x=np.load("data/helix/pointnext/raw/Area_1225_charleston_lvl2_41_1.npy",allow_pickle=True)[:, -1]
unique, counts = np.unique(x, return_counts=True)
print(np.asarray((unique, counts)).T)

[[0.00000e+00 3.25810e+04]
 [1.00000e+00 1.06091e+05]
 [2.00000e+00 3.53294e+05]
 [3.00000e+00 1.38248e+05]
 [4.00000e+00 4.45430e+04]
 [5.00000e+00 9.97130e+04]
 [6.00000e+00 4.81500e+03]]


In [7]:
x

array([6., 6., 6., ..., 1., 1., 1.])

In [13]:
raw_samples = glob.glob("data/helix/pointnext/raw/*.npy")
num_per_class = np.zeros(9)
for sample in raw_samples:
    labels = np.load(sample, allow_pickle=True)[:, -1]
    num_per_class += np.bincount(labels.astype(int), minlength=9)
num_per_class.astype(np.int32)

array([ 68424220, 109636976, 170714860,  19029068,  14087767,  97219579,
         6693202,   2522767,  10988524], dtype=int32)

In [41]:
csv_files = glob.glob("data/helix/raw/data/*/*.csv")
num_per_class = { key : 0 for key in range(1, 20) }
num_per_class = pd.Series(num_per_class).to_frame()
for file in csv_files:
    df = pd.read_csv(file)
    df['Classification'] = df['Classification'].astype(int)
    num_labels = df['Classification'].value_counts().to_frame().astype(int)
    num_per_class = pd.concat((num_per_class, num_labels), axis=1).sum(axis=1).astype(int)
num_per_class

1        759444
2       3227826
3     106409150
4      12835791
5      19029068
6         98044
7       9130843
8      97219579
9      14908023
10      6693202
11     10427236
12     58533933
13      1685304
14      1797558
15            0
16      2522767
17        79625
18    142971046
19     10988524
dtype: int64